In [2610]:
# Standard library imports
import os
import sys
import re
import warnings
import random
import hashlib

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning and preprocessing
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler  # Assuming you might need it

# Specific models and tools
from xgboost import XGBClassifier
import xgboost as xgb

# Encoding and feature selection
from category_encoders import TargetEncoder  # Fixed the import based on usage
from scipy.stats import randint, uniform

# Model persistence
from joblib import dump, load

# Miscellaneous settings
%matplotlib inline
warnings.filterwarnings('ignore')


In [2611]:
content = "big5"

In [2612]:
# Load the processed data csv into a DataFrame
df = pd.read_csv(f'data/processed/processed_data_{content}.csv')

In [2613]:
# Parse the date_temp column, which is in YYYYMMDD format, into a datetime object, and store in a new column 'date_temporary'
df['date_temporary'] = pd.to_datetime(df['Date'], format='%Y%m%d')

### Date settings

In [2614]:
import pandas as pd

# Get the current date dynamically
date_today = pd.Timestamp.now().normalize()  # .normalize() sets the time to 00:00:00

# Calculate the date 2 weeks ago from the current date
date_2_weeks_ago = date_today - pd.DateOffset(weeks=4)

date_cutoff = date_today - pd.DateOffset(days=1000)

In [2615]:
# Delete all rows where the date_temporary column is older than date_cutoff
df = df[df['date_temporary'] >= date_cutoff]

In [2616]:
#df_validationset = df.tail(250)
#df = df.iloc[:-250]

# define df_validationset as all the rows in df where the date_temporary column is greater than date_2_weeks_ago
df_validationset = df[df['date_temporary'] > date_2_weeks_ago]

# define df as all the rows in df where the date_temporary column is less than or equal to date_2_weeks_ago
df = df[df['date_temporary'] <= date_2_weeks_ago]

In [2617]:
len(df), len(df_validationset)

(6430, 180)

In [2618]:
# Drop the date_temporary column
df.drop(columns=['date_temporary'], inplace=True)
df_validationset.drop(columns=['date_temporary'], inplace=True)

In [2619]:
import ast
import pandas as pd

teams_dict = {}

# Assuming 'teams_dict.txt' contains the dictionary as a single string
with open('data/teams_dict.txt', 'r') as file:
    # Read the entire file content into a single string
    data = file.read()
    # Safely evaluate the string as a Python dictionary
    teams_dict = ast.literal_eval(data)

In [2620]:
# Sort the df and df_validationset DataFrames by the 'Date', 'Div', 'Time' columns
df.sort_values(['Date', 'Div', 'Time'], inplace=True)
df_validationset.sort_values(['Date', 'Div', 'Time'], inplace=True)

# Set the 'Date' and 'FTR2' column as the index
df.set_index(['Date', 'FTR2'], inplace=True)
df_validationset.set_index(['Date', 'FTR2'], inplace=True)

In [2621]:
#import train_test_split
from sklearn.model_selection import train_test_split

# Split the data into X and y
X = df.drop('FTR', axis=1)
y = df['FTR']

X.columns = [re.sub(r'[<]', '_st_', str(col)) for col in X.columns]
X.columns = [re.sub(r'[>]', '_gt_', str(col)) for col in X.columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2622]:
len(X_train), len(X_test)

(5144, 1286)

In [2623]:
# Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier


# Create a pipeline
pipeline = Pipeline([
    ('target_encoder', TargetEncoder()),
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier())
])

# Define the hyperparameters

param_distributions = {

    'target_encoder__smoothing': randint(1, 100),
    'xgb__n_estimators': randint(100, 1000),
    'xgb__max_depth': randint(3, 10),
    'xgb__learning_rate': uniform(0.01, 0.6),
    'xgb__subsample': uniform(0.3, 0.7),
    'xgb__colsample_bytree': uniform(0.3, 0.7),
    'xgb__gamma': randint(0, 5),
    'xgb__reg_alpha': uniform(0, 1),
    'xgb__reg_lambda': uniform(0, 1),
    'xgb__min_child_weight': randint(1, 10),
    'xgb__scale_pos_weight': uniform(0.5, 1.5)
  
}

# Create a RandomizedSearchCV object
search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=5,
    cv=TimeSeriesSplit(n_splits=5),
    scoring='f1',
    verbose=1,
    n_jobs=-1,
    random_state=42
)

# Fit the model
search.fit(X_train, y_train)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Pipeline(steps=[('target_encoder',
                                              TargetEncoder()),
                                             ('scaler', StandardScaler()),
                                             ('xgb',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            device=None,
                                                            early_stopping_rounds=None...
                                        'xgb__reg_alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001693ADFEDE0>,
                                        'xgb__reg_lambda': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000169B15349E0>,
                                        'xgb__scale_pos_weight': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000169973F3DD0>,
                                        'xgb__subsample': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x000001699EF94470>},
                   random_state=42, scoring='f1', verbose=1)

In [2624]:
# print the classification report
print(classification_report(y_test, search.predict(X_test)))

              precision    recall  f1-score   support

           0       0.67      0.57      0.62       717
           1       0.54      0.64      0.59       569

    accuracy                           0.60      1286
   macro avg       0.61      0.61      0.60      1286
weighted avg       0.61      0.60      0.60      1286



In [2625]:
def generate_sliding_windows(X, window_size, step):
    n_samples = len(X)
    windows = []
    for start_idx in range(0, n_samples - window_size + 1, step):
        end_idx = start_idx + window_size
        if end_idx > n_samples:
            break  # Avoid going beyond the dataset
        train_indices = list(range(max(0, start_idx - window_size), start_idx))
        test_indices = list(range(start_idx, end_idx))
        windows.append((train_indices, test_indices))
    return windows

negative_count = len(df[df['FTR'] == 0])
positive_count = len(df[df['FTR'] == 1])
scale_pos_weight_value = negative_count / positive_count

# Define the hyperparameter search space
param_dist = {
    
    #'xgb__clf__max_depth': [1,2,3],
    #'xgb__clf__learning_rate': [0.001, 0.01, 0.1],
    #'xgb__clf__lambda': [1, 1.5, 2],  # L2 regularization term on weights
    #'xgb__clf__alpha': [0, 0.5, 1],  # L1 regularization term on weights
    #'xgb__clf__n_estimators': [1, 5, 100],

    #'rf__clf__max_depth': [None, 4, 6],
    #'rf__clf__min_samples_split': [2, 5],
    #'rf__clf__min_samples_leaf': [1, 2],
    #'rf__clf__bootstrap': [True, False],
    #'rf__clf__n_estimators': [50, 100, 200],

    'lr__clf__C': [0.1, 1, 10],  # Inverse of regularization strength; smaller values specify stronger regularization.
    'lr__clf__penalty': ['l1', 'l2', 'elasticnet'],  # Specify the norm of the penalty.
    'lr__clf__solver': ['saga'],  # Algorithm to use in the optimization problem, 'saga' supports all penalties.
    'lr__clf__l1_ratio': [0.5],  # The Elastic-Net mixing parameter, with 0 <= l1_ratio <= 1. Only used if penalty='elasticnet'.

    #'cat__clf__depth': [1,2,3,4],
    #'cat__clf__learning_rate': [0.01, 0.05, 0.1],
    #'cat__clf__iterations': [50, 100, 200],
    #'cat__clf__l2_leaf_reg': [1, 3, 5],

    'gb__clf__learning_rate': [0.01, 0.1, 0.2],
    'gb__clf__n_estimators': [50, 100, 200],
    'gb__clf__max_depth': [3, 5, 7],
    'gb__clf__min_samples_split': [2, 5],
    'gb__clf__min_samples_leaf': [1, 2],

}


param_test = {
    
    'xgb__clf__max_depth': [1,2,3],
    'xgb__clf__learning_rate': [0.001, 0.01, 0.1],
    'xgb__clf__lambda': [1, 1.5, 2],  # L2 regularization term on weights
    'xgb__clf__alpha': [0, 0.5, 1],  # L1 regularization term on weights
    'xgb__clf__n_estimators': [1, 5, 100],


}

In [2626]:
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Define a custom scoring function
def xgb_early_stopping_score(y, estimator, X, y_true, sample_weight=None):
    """
    Custom scorer that uses early stopping.
    """
    # Split X into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y_true, test_size=0.2, random_state=42)
    
    # Fit with early stopping
    eval_set = [(X_val, y_val)]
    estimator.fit(X_train, y_train, early_stopping_rounds=10, eval_set=eval_set, verbose=False)
    
    # Predict on the validation set
    y_pred = estimator.predict(X_val)
    
    # Return the F1 score
    return f1_score(y_val, y_pred, pos_label=1)

In [2627]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split, StratifiedKFold, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import make_scorer, f1_score, classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE
import numpy as np
import pandas as pd
from imblearn.pipeline import Pipeline as ImbPipeline
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# LightGBM
from lightgbm import LGBMClassifier

# naive bayes
from sklearn.naive_bayes import GaussianNB

#catboost
from catboost import CatBoostClassifier

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier

# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Logistic Regression
from sklearn.linear_model import LogisticRegression

# Define the F1 score for the '1' class
f1_scorer = make_scorer(f1_score, pos_label=1)

best_f1_score = 0
best_f1_params = None
best_window_size = None
best_precision = 0
best_model = None 
f1_scores = []
precision_scores = []



# Make the custom scorer
custom_scorer = make_scorer(xgb_early_stopping_score, greater_is_better=True, needs_proba=False, X=X, y_true=y)

# Set the window_size and step to 5% of the dataset
window_size = int(len(X) * 0.1)
step = int(len(X) * 0.1)

# Initialize an empty list to store precision scores
precision_scores = []

# Initialize an empty dataframe to store misclassified samples
misclassified_samples = pd.DataFrame(columns=X.columns)

# Generate windows
window_splits = generate_sliding_windows(X, window_size, step)

# Initialize training indices with the first window
train_end_index = window_size

# Iterate over each sliding window
for i, (train_index, test_index) in enumerate(window_splits):

    # Update training indices to include the next window
    train_index = list(range(train_end_index))
    train_end_index += window_size

    X_train, y_train = X.iloc[train_index], y.iloc[train_index]
    X_test, y_test = X.iloc[test_index], y.iloc[test_index]

    print(f"Iteration {i+1} Training Data Shape: {X_train.shape}")

    # Combine misclassified samples from previous iterations with current training data
    if not misclassified_samples.empty:
        X_train_combined = pd.concat([X_train, misclassified_samples[X_train.columns]], axis=0)
        y_train_combined = pd.concat([y_train, misclassified_samples['FTR']], axis=0)
    else:
        X_train_combined = X_train
        y_train_combined = y_train

    # Calculate misclassification frequency
    misclassified_freq = y_train_combined.value_counts(normalize=True)

    # Define class weights based on misclassification frequency
    class_weights = {0: 1, 1: max(0.6, 1 - misclassified_freq.get(1, 0.5))}  # Adjust dynamically to penalize misclassification of class 1 more heavily

    # Define pipelines for each classifier with SMOTE and TargetEncoder
    pipeline_xgb = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', XGBClassifier(random_state=42, verbose=0))
    ])

    pipeline_gb = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', GradientBoostingClassifier(random_state=42, verbose=0))
    ])

    # pipeline for logistic regression
    pipeline_lr = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', LogisticRegression(random_state=42, verbose=0))
    ])

    # pipeline for catboost classifier
    pipeline_cat = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', CatBoostClassifier(random_state=42, verbose=0))
    ])

    # pipeline for random forest
    pipeline_rf = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', RandomForestClassifier(random_state=42, verbose=0))
    ])

    # LightGBM pipeline
    pipeline_lgbm = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', LGBMClassifier(random_state=42, force_col_wise='true', verbose=0))
    ])

    # Adaboost pipeline
    pipeline_ada = ImbPipeline([
        ('target_encoder', TargetEncoder()),
        ('smote', SMOTE(random_state=42)),
        ('clf', AdaBoostClassifier(random_state=42))
    ])

    # Combine them into an ensemble classifier
    ensemble_clf = VotingClassifier(estimators=[
        ('xgb', pipeline_xgb),
        ('gb', pipeline_gb),
        ('lr', pipeline_lr),
        #('cat', pipeline_cat),
        #('rf', pipeline_rf),
        ('lgbm', pipeline_lgbm),
        ('ada', pipeline_ada)
    ], voting='soft')

    # Setup RandomizedSearchCV
    clf = RandomizedSearchCV(
        estimator=ensemble_clf,
        param_distributions=param_dist,
        #param_distributions=param_test,
        n_iter=2,
        scoring=f1_scorer,
        #scoring='precision',
        cv=TimeSeriesSplit(n_splits=2),
        random_state=42,
        n_jobs=-1,
        verbose=0
    )  


    # Fit RandomizedSearchCV
    clf.fit(X_train_combined, y_train_combined)

    # Get the best parameters
    best_params = clf.best_params_
    print("Best Parameters:", best_params)

    # Use the best estimator
    best_pipe = clf.best_estimator_

    # Make predictions
    y_proba = best_pipe.predict_proba(X_test)

    # Apply threshold
    threshold = 0.5  # You can adjust this threshold as needed
    y_pred = (y_proba[:, 1] >= threshold).astype(int)

    current_f1_score = f1_score(y_test, y_pred, pos_label=1)

    if current_f1_score > best_f1_score:
        best_f1_score = current_f1_score
        best_f1_params = clf.best_params_
        #best_model = clf.best_estimator_ 

    # ------------------------------------------------

    best_model = clf.best_estimator_ 

    # Calculate precision score
    precision = np.mean(y_test == y_pred)
    precision_scores.append(precision)
    print("Precision:", precision)

    print()

# Print the best F1 score and its corresponding parameters
print()
print("Best F1 Score:", best_f1_score)

# print the classification report of the best model on the full dataset
print(classification_report(y_test, y_pred))

Iteration 1 Training Data Shape: (643, 9)
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [2628]:
# Correlation matrix
corr = df.corr()

# Put the target column to the front
cols = list(corr.columns)
cols.insert(0, cols.pop(cols.index('FTR')))
corr = corr.loc[cols, cols]

# Plot the correlation matrix
#plt.figure(figsize=(10, 8))
#sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=2)
#plt.show()



In [2629]:
# Print the feature importances
importances = search.best_estimator_.named_steps['xgb'].feature_importances_
features = X_train.columns
importances_df = pd.DataFrame({'features': features, 'importances': importances})
importances_df = importances_df.sort_values('importances', ascending=False)

# Plot the feature importances
#plt.figure(figsize=(10, 8))
#sns.barplot(x='importances', y='features', data=importances_df)
#plt.title('Feature Importances')
#plt.show()



### Validation

In [2630]:
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

df_val = df_validationset.copy()

# Calculate the predicted probabilities for the validation set
y_val_proba = best_model.predict_proba(df_val.drop(columns=['FTR']))

# Initialize variables to track the best threshold and its corresponding accuracy
best_threshold = 0.5
best_accuracy = 0

# Iterate over potential threshold values
for threshold in np.arange(0.5, 0.85, 0.001):
    # Apply the current threshold to generate predictions
    y_val_pred = (y_val_proba[:, 1] >= threshold).astype(int)
    
    # Evaluate accuracy for the current set of predictions
    accuracy = accuracy_score(df_val['FTR'], y_val_pred)
    
    # Update the best threshold and accuracy as needed
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold 

# Print the best threshold and its accuracy
print(f"Best Threshold: {best_threshold}")
print(f"Best Accuracy: {best_accuracy}")

# Apply the best threshold to generate the final set of predictions
y_val_pred_best = (y_val_proba[:, 1] >= best_threshold).astype(int)

# Add the prediction probabilities and final predictions to df_val
#df_val['proba_0'] = y_val_proba[:, 0].round(3)
df_val['Probability'] = y_val_proba[:, 1].round(3)
df_val['Prediction'] = y_val_pred_best

# Ensure y_val_pred_best is generated as before with the best threshold applied

# Directly filter df_val and add necessary columns without separate reindexing steps
filtered_df_val = df_val[df_val['Probability'] > best_threshold].copy()

filtered_df_val.reset_index(inplace=True)

filtered_df_val['Prediction'] = (filtered_df_val['Probability'] >= best_threshold).astype(int)
#filtered_df_val['Actual Result'] = filtered_df_val['FTR']
filtered_df_val['Correct Prediction'] = (filtered_df_val['Prediction'] == filtered_df_val['FTR']).astype(bool)

index_to_team = {v: k for k, v in teams_dict.items()}
filtered_df_val['Team'] = filtered_df_val['Team_ID'].map(index_to_team)
filtered_df_val['Opponent'] = filtered_df_val['Opp_ID'].map(index_to_team)


display_columns = [
    'Date', 'Team', 'Opponent', 'Probability', 'Prediction',
    'FTR', 'FTR2', 'Correct Prediction', 'AvgH', 'AvgD', 'AvgA'
]



Best Threshold: 0.562
Best Accuracy: 0.7055555555555556


In [2631]:
output = filtered_df_val[display_columns]


output = output.sort_values('Date', ascending=False)

output = output[output['Probability'] > best_threshold].copy()

In [2632]:
display(output)

,Date,Team,Opponent,Probability,Prediction,FTR,FTR2,Correct Prediction,AvgH,AvgD,AvgA
48,20240401,Inter,Empoli,0.656,1,1,1,True,1.20,7.04,13.25
47,20240401,Bologna,Salernitana,0.678,1,1,1,True,1.29,5.44,11.25
46,20240401,Leeds,Hull,0.663,1,1,1,True,1.43,4.75,6.98
45,20240401,Middlesbrough,Sheffield Weds,0.570,1,1,1,True,1.80,3.76,4.32
44,20240401,Coventry,Cardiff,0.637,1,0,2,False,1.53,4.28,6.00
43,20240401,Leicester,Norwich,0.690,1,1,1,True,1.65,4.17,4.83
42,20240331,Real Madrid,Ath Bilbao,0.667,1,1,1,True,1.55,4.29,5.87
41,20240331,Man City,Arsenal,0.610,1,0,0,False,1.92,3.80,3.86
40,20240331,Liverpool,Brighton,0.720,1,1,1,True,1.37,5.63,7.37
39,20240331,Bochum,Darmstadt,0.657,1,0,0,False,1.65,4.18,5.06


### Results

In [2633]:
# Display the Correct Prediction True / False ratio, and ther percentage of correct predictions
correct_predictions = output['Correct Prediction'].sum()
total_predictions = len(output)
correct_ratio = correct_predictions / total_predictions

print(f"Best Threshold: {best_threshold:.2f}")
print(f"Best Accuracy: {best_accuracy:.2f}")
print()
print(f"Total Predictions: {total_predictions}")
print(f"Total Correct Predictions: {correct_predictions}")
print()
print(f"Percentage of Correct Predictions: {correct_ratio * 100:.2f}%")

Best Threshold: 0.56
Best Accuracy: 0.71

Total Predictions: 49
Total Correct Predictions: 35

Percentage of Correct Predictions: 71.43%


In [2634]:
# Timestamp
import datetime

# Get the current date and time
now = datetime.datetime.now()

# Format the current date and time as a string
timestamp = now.strftime("%Y%m%d_%H%M%S")

# save filtered_df_val[display_columns] to a CSV file
output.to_csv(f'data/predictions/predictions_{content}_{timestamp}.csv', index=False)

In [2635]:
import winsound
frequency = 400  # Set Frequency To 2500 Hertz
duration = 200  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)